Collaborative Filtering

In [0]:
## 01 - Install Spark Packages
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import *
from pyspark import *
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
import sys
import pandas as pd
import seaborn as sns
from time import time
from pyspark.sql import Row
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel

In [0]:
df_output = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/kkctong@connect.ust.hk/outputdata.json")
df_output.show()

+----------+--------------------+--------------------+------------+-----+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+
 asin| cleaned_words| features|indexedLabel|label|pre_label|prediction| probability| rawFeatures| rawPrediction| reviewText| reviewerID| words|
+----------+--------------------+--------------------+------------+-----+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+
B00002R28C|[seeing, ffi, iii...|[[2708, 15931, 23...| 0.0| 1.0| 4.0| 1.0|[1, [0.0889260966...|[[2708, 15931, 23...|[1, [-2.326818366...|Seeing as I had F...|A103B6MQ5IF2BK|[seeing, as, i, h...|
B00004SVRR|[big, fan, came, ...|[[734, 780, 10157...| 0.0| 1.0| 4.0| 1.0|[1, [0.0853294299...|[[734, 780, 10157...|[1, [-2.372044555...|I was a big fan w...|A103QIFFX532JU|[i, was, a, big, ...|
B00004SVXA|[ah, nes, revolut...|[[4416, 4527, 485...| 0.0| 1.0| 5.0| 1.0|[1, [0.0962394833...|[[4416, 4527, 485...|[1, [-2.239724707...|Ah, The NES, a re...|A105GCSUC8LQ1Z|[ah, the, nes, a,...|
B00004TJCL|[counter, strike,...|[[524, 594, 2334,...| 1.0| 0.0| 1.0| 1.0|[1, [0.0593518936...|[[524, 594, 2334,...|[1, [-2.763085085...|Counter-Strike is...|A105GWGM7PDAI2|[counter, strike,...|
B00004YRQA|[love, memory, ca...|[[1024, 1434, 508...| 0.0| 1.0| 5.0| 1.0|[1, [0.0724859951...|[[1024, 1434, 508...|[1, [-2.549114521...|I love this memor...|A105GWGM7PDAI2|[i, love, this, m...|
B00004U55G|[call, power, ii,...|[[48, 734, 1344, ...| 0.0| 1.0| 4.0| 1.0|[1, [0.0617660964...|[[48, 734, 1344, ...|[1, [-2.720644669...|Call to Power II ...|A1085T5S1400VA|[call, to, power,...|
B00004U8KO|[tropico, fun, ga...|[[62, 734, 1024, ...| 0.0| 1.0| 5.0| 1.0|[1, [0.0878090741...|[[62, 734, 1024, ...|[1, [-2.340684470...|Tropico is a fun ...|A1085T5S1400VA|[tropico, is, a, ...|
B00000DMAU|[far, one, best, ...|[[301, 2083, 1053...| 0.0| 1.0| 5.0| 1.0|[1, [0.0856198516...|[[301, 2083, 1053...|[1, [-2.368329233...|this by far one o...|A10ABCICVBQW92|[this, by, far, o...|
B00004WHWG|[never, made, mac...|[[8344, 10281, 12...| 1.0| 0.0| 1.0| 1.0|[1, [0.0889260966...|[[8344, 10281, 12...|[1, [-2.326818366...|they never made a...|A10DIDVYDN2Q5Z|[they, never, mad...|
B00000F1GM|[one, launch, tit...|[[2007, 2126, 293...| 0.0| 1.0| 4.0| 1.0|[1, [0.0731554098...|[[2007, 2126, 293...|[1, [-2.539199822...|One of the launch...|A10EAONPHFORLH|[one, of, the, la...|
B00002SW8P|[original, nes, o...|[[5078, 6140, 945...| 0.0| 1.0| 4.0| 1.0|[1, [0.0792972393...|[[5078, 6140, 945...|[1, [-2.451933933...|I own the origina...|A10EM4DKTUAR7W|[i, own, the, ori...|
B00002SWA2|[know, came, one,...|[[780, 1805, 2708...| 0.0| 1.0| 4.0| 1.0|[1, [0.0962394833...|[[780, 1805, 2708...|[1, [-2.239724707...|I know they came ...|A10EM4DKTUAR7W|[i, know, they, c...|
B000035Y4T|[sequal, continue...|[[2083, 4828, 945...| 0.0| 1.0| 4.0| 1.0|[1, [0.0852612174...|[[2083, 4828, 945...|[1, [-2.372918850...|The sequal is wha...|A10EM4DKTUAR7W|[the, sequal, is,...|
B00004YRQ9|[definitely, best...|[[3349, 3805, 467...| 0.0| 1.0| 5.0| 1.0|[1, [0.0671521602...|[[3349, 3805, 467...|[1, [-2.631281007...|Definitely the be...|A10EN9IKKOEY97|[definitely, the,...|
B00000J97G|[origanal, gamebo...|[[855, 2083, 2129...| 0.0| 1.0| 5.0| 1.0|[1, [0.1274926017...|[[855, 2083, 2129...|[1, [-1.923312795...|This is the origa...|A10FCZHS6L5SA1|[this, is, the, o...|
B00002SU5A|[huge, fan, vampi...|[[42, 150, 197, 2...| 1.0| 0.0| 2.0| 1.0|[1, [0.4785949929...|[[42, 150, 197, 2...|[1, [-0.085672391...|I am a huge fan o...|A10FEGI2FQD7L9|[i, am, a, huge, ...|
B00002SU5D|[yes, best, rts, ...|[[594, 780, 855, ...| 0.0| 1.0| 5.0| 1.0|[1, [0.0857803149...|[[594, 780, 855, ...|[1, [-2.366281347...|Yes, this is THE ...|A10JPTQTDNTV43|[yes, this, is, t...|
B00004T77A|[believe, game, b...|[[223, 858, 2083,...| 0.0| 1.0| 5.0| 1.0|[1, [0.0797996837...|[[223, 858, 2083,...|[1, [-2.44507

In [0]:
# drop not needed columns
df = df_output.drop("_corrupt_record", "cleaned_words", "label", "features", "indexedLabel","prediction", "probability", "rawFeatures", "rawPrediction", "words", 'reviewText','reviewTime','reviewerName','unixReviewTime','summary')
df = df.selectExpr("asin as item","pre_label as rating", "reviewerID")
df.count()

Out[11]: 125210

In [0]:
df.describe().show()

+-------+--------------------+-----------------+--------------------+
summary| item| rating| reviewerID|
+-------+--------------------+-----------------+--------------------+
 count| 125210| 125210| 125210|
 mean| 2.490072014274827E9|4.423312834438144| null|
 stddev|2.5459666225318317E9|1.041619473207905| null|
 min| 0439893577| 1.0|A00046902LP5YSDV0...|
 max| B00LFPS0CY| 5.0| AZZYW4YOE1B6E|
+-------+--------------------+-----------------+--------------------+

In [0]:
#Turn itemID and reviewerID to string
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df).transform(df)
transformed.show()

(df_train,df_test)=transformed.randomSplit([0.8, 0.2])
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="reviewerID_index",itemCol="item_index",ratingCol="rating",coldStartStrategy="drop",
        nonnegative=True)
model=als.fit(df_train)

+----------+------+--------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+--------------+----------+----------------+
B00002R28C| 4.0|A103B6MQ5IF2BK| 1231.0| 2588.0|
B00004SVRR| 4.0|A103QIFFX532JU| 7189.0| 15087.0|
B00004SVXA| 5.0|A105GCSUC8LQ1Z| 23313.0| 15092.0|
B00004TJCL| 1.0|A105GWGM7PDAI2| 1936.0| 15093.0|
B00004YRQA| 5.0|A105GWGM7PDAI2| 216.0| 15093.0|
B00004U55G| 4.0|A1085T5S1400VA| 15084.0| 7353.0|
B00004U8KO| 5.0|A1085T5S1400VA| 7205.0| 7353.0|
B00000DMAU| 5.0|A10ABCICVBQW92| 14850.0| 15106.0|
B00004WHWG| 1.0|A10DIDVYDN2Q5Z| 10111.0| 31714.0|
B00000F1GM| 4.0|A10EAONPHFORLH| 310.0| 15124.0|
B00002SW8P| 4.0|A10EM4DKTUAR7W| 23229.0| 4138.0|
B00002SWA2| 4.0|A10EM4DKTUAR7W| 14988.0| 4138.0|
B000035Y4T| 4.0|A10EM4DKTUAR7W| 4220.0| 4138.0|
B00004YRQ9| 5.0|A10EN9IKKOEY97| 496.0| 7365.0|
B00000J97G| 5.0|A10FCZHS6L5SA1| 2744.0| 31728.0|
B00002SU5A| 2.0|A10FEGI2FQD7L9| 4215.0| 31729.0|
B00002SU5D| 5.0|A10JPTQTDNTV43| 7168.0| 1175.0|
B00004T77A| 5.0|A10JPTQTDNTV43| 15064.0| 1175.0|
B00002R28C| 4.0|A10MJA08WPGOM0| 1231.0| 31767.0|
B00001OWYW| 5.0|A10VQUCFXVTXHG| 1054.0| 15167.0|
+----------+------+--------------+----------+----------------+
only showing top 20 rows

In [0]:
#Build ALS model
(df_train,df_test)=transformed.randomSplit([0.8, 0.2])
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="reviewerID_index",itemCol="item_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
model=als.fit(df_train)

In [0]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(df_test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

RMSE=1.9760079696328077
+----------+------+--------------+----------+----------------+----------+
 item|rating| reviewerID|item_index|reviewerID_index|prediction|
+----------+------+--------------+----------+----------------+----------+
B00004YRQ9| 2.0|A2X85UOCANKYCH| 496.0| 11289.0| 2.5446064|
B00005YTYJ| 2.0| ARA9XGPNOXSQO| 1238.0| 7143.0| 2.5605683|
B00004YRQ9| 5.0|A1I1FB6YT9OV03| 496.0| 17335.0| 2.7818017|
B00004YRQ9| 1.0|A3O12K1YQR2FZ9| 496.0| 6409.0| 2.1293063|
B00004YRQ9| 5.0|A22TL53EPV6LII| 496.0| 1954.0| 3.3790317|
B00004YRQ9| 5.0|A2QE3WKYWU0EXU| 496.0| 2124.0| 2.3926754|
B00005YTYJ| 1.0| AMPCEPYM5RERK| 1238.0| 29985.0| 1.5026926|
B00006F7RS| 1.0|A2J57EQ9KPGXR9| 10230.0| 192.0|0.42970514|
B000035XR9| 5.0| A93EK1533EVAD| 4219.0| 28350.0| 2.3305666|
B000035Y0J| 5.0|A2ICHQXE5HML2E| 15003.0| 10465.0| 3.7227323|
B000035XR9| 4.0|A2WTZKL94OCH5E| 4219.0| 3358.0| 3.3092873|
B00002SU5D| 2.0|A281L0GOXH4HQU| 7168.0| 5232.0| 1.1922174|
B00005T7ZP| 4.0|A3PASG15BRR40D| 15254.0| 42.0| 2.0400417|
B0006GBD04| 4.0|A1PSOVIQK992SV| 2821.0| 1874.0| 4.5868435|
B00006JC3S| 4.0|A18AQ3EFPYBKH0| 2776.0| 1760.0| 2.4457407|
B00000DMAX| 5.0|A1LBAC84TLIGAX| 1650.0| 25.0| 4.3366833|
B00000DMAX| 5.0|A2WTO0ST2SUUY9| 1650.0| 23490.0| 4.1876755|
B00000DMAX| 5.0|A1BOGEQ3FLZDKY| 1650.0| 8000.0| 3.7253869|
B0002ILS1U| 5.0|A3NWDVRYFBV72W| 1975.0| 61.0| 3.6696882|
B00002SVFR| 5.0| AJM37F8PBJXUX| 2748.0| 610.0| 3.9787762|
+----------+------+--------------+----------+----------------+----------+
only showing top 20 rows

In [0]:
user_recs=model.recommendForAllUsers(20).show(10,False)

+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
reviewerID_index|recommendations |
+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
31 |[[11730, 5.8071594], [7284, 5.799088], [2943, 5.781532], [6327, 5.757176], [3153, 5.742767], [3803, 5.7170877], [2630, 5.682657], [7599, 5.668342], [8643, 5.664221], [3536, 5.6458874], [7020, 5.637905], [29085, 5.6203303], [7222, 5.6108623], [2488, 5.591745], [3741, 5.570154], [10962, 5.558677], [11875, 5.556525], [20765, 5.552726], [9288, 5.552129], [15061, 5.5466466]] |
34 |[[16395, 5.8907723], [17804, 5.782589], [18550, 5.7468567], [10689, 5.7046905], [21609, 5.6449313], [11866, 5.637377], [13655, 5.6100407], [4232, 5.5979877], [25563, 5.554622], [16936, 5.547292], [10336, 5.544349], [11971, 5.5136905], [16423, 5.50756], [5085, 5.485399], [14737, 5.4636974], [16269, 5.4636235], [23555, 5.4613132], [27445, 5.4273515], [1730, 5.4207363], [12469, 5.4179373]]|
53 |[[7799, 7.6329365], [27971, 7.630368], [9334, 7.4583], [17270, 7.353196], [7237, 7.1818943], [922, 7.17661], [10175, 7.130026], [1428, 7.087201], [9907, 7.0814133], [3977, 7.038331], [9769, 7.029687], [9493, 7.0211], [14547, 6.981402], [13647, 6.9757996], [4006, 6.9669304], [28732, 6.962822], [28097, 6.9588275], [8479, 6.9544325], [9148, 6.948567], [14774, 6.9475875]] |
65 |[[27592, 6.503985], [14269, 6.381446], [22465, 6.3075457], [22737, 6.2006125], [13994, 6.168951], [24958, 6.14637], [12708, 6.1244755], [33817, 6.0402308], [8322, 6.0360446], [32975, 5.960906], [17176, 5.9375257], [28883, 5.9186883], [13655, 5.9123917], [18599, 5.863477], [27445, 5.810512], [26415, 5.692199], [12437, 5.637991], [17641, 5.603272], [9369, 5.5979023], [22484, 5.5919476]] |
78 |[[8219, 6.323934], [9564, 6.282268], [3641, 6.2653875], [2390, 6.2614374], [11521, 6.1994777], [21863, 6.1420674], [11674, 6.1259356], [8223, 6.103153], [4006, 6.100485], [12099, 6.094174], [5559, 6.0810385], [21246, 6.0800915], [8855, 6.076026], [3977, 6.0717854], [12095, 6.066253], [5385, 6.0482416], [9297, 6.047966], [24203, 6.044449], [2591, 6.011059], [18423, 6.008684]] |
85 |[[9148, 6.365312], [5009, 6.300527], [2943, 6.297328], [5129, 6.243665], [1965, 6.230893], [6487, 6.2287745], [11119, 6.217116], [6694, 6.201874], [7147, 6.1922073], [21212, 6.1852074], [3868, 6.174736], [4693, 6.1744723], [3153, 6.157646], [2809, 6.1484904], [3762, 6.135463], [4274, 6.1306415], [7284, 6.126667], [6598, 6.112607], [3201, 6.1113753], [2330, 6.110042]] |
108 |[[3377, 5.9492087], [6567, 5.821794], [9148, 5.7961626], [13000, 5.7934093], [10539, 5.793177], [4102, 5.792871], [18299, 5.785081], [27357, 5.73757], [24820, 5.73757], [12985, 5.7027736], [3347, 5.6866436], [7078, 5.662111], [20766, 5.6103635], [21212, 5.600982], [11106, 5.5446124], [10743, 5.5374107], [2296, 5.5133886], [20122, 5.511362], [16489, 5.494685], [27502, 5.471687]] |
133 |[[3377, 5.6765018], [2086, 5.3810453], [15617, 5.3793483], [10106, 5.338496], [19614, 5.328906], [19477, 5.303338], [21189, 5.285963], [13340, 5.2732897], [9148, 5.272633], [2493, 5.253926], [3094, 5.227382], [4919, 5.2253833], [7078, 5.2215123], [10493, 5.212589], [7237, 5.1893477], [3353, 5.161996], [7799, 5.1475096], [21045, 5.1420646], [3373, 5.124006], [31855, 5.1224213]] |
137 |[[9476, 6.1674695], [32069, 6.0232725], [13655, 5.9837756], [2

In [0]:
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['reviewerID_index'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna() 
nrecs=nrecs.sort_values('reviewerID_index')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['reviewerID_index']], axis = 1)
nrecs.columns = [
        
        'item_index',
        'rating',
        'reviewerID_index'
       
     ]
md=transformed.select(transformed['reviewerID'],transformed['reviewerID_index'],transformed['item'],transformed['item_index'])
md=md.toPandas()
dict1 =dict(zip(md['reviewerID_index'],md['reviewerID']))
dict2=dict(zip(md['item_index'],md['item']))
nrecs['reviewerID']=nrecs['reviewerID_index'].map(dict1)
nrecs['item']=nrecs['item_index'].map(dict2)
nrecs=nrecs.sort_values('reviewerID')
nrecs.reset_index(drop=True, inplace=True)
new=nrecs[['reviewerID','item','rating']]
new['recommendations'] = list(zip(new.item, new.rating))
res=new[['reviewerID','recommendations']]  
res_new=res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()
print(res_new)

reviewerID recommendations
0 A00046902LP5YSDV0VVNF [(B001CSPTSC, 2.440577268600464), (B0015KXFB0,...
1 A00263941WP7WCIL7AKWL [(B008Y2UKF6, 3.4636950492858887), (B008GPMF9A...
2 A0029274J35Q1MYNKUWO [(B009RWMCY4, 4.639435291290283), (B004W2KRP0,...
3 A004507634IQ8TNQ3YTUI [(B000HBGSMQ, 4.533412456512451), (B0031EQHJU,...
4 A005481137I9SCAWEF7ON [(B0014SN568, 6.02885103225708), (B00CEQ1HPK, ...
... ... ...
51081 AZZLI36GZV6ZD [(B00591LHZ2, 4.296446800231934), (B005VGSX3E,...
51082 AZZNK89PXD006 [(B001KW063S, 4.457386493682861), (B003IT6E8C,...
51083 AZZT1ERHBSNQ8 [(0615444172, 5.718450546264648), (B008N6INE8,...
51084 AZZTC2OYVNE2Q [(B00000IWCX, 4.83568000793457), (B00BD9OLW0, ...
51085 AZZYW4YOE1B6E [(B00000K1VK, 5.421123027801514), (B004BWTO4G,...

[51086 rows x 2 columns]

In [0]:
#Relationship between user with common items reviewed
item_0 = transformed.filter("item = 'B001HBHNHE'")
item_1 = transformed.filter("item = 'B001T7QJ9O'")

In [0]:
item_0 = transformed.filter("item = 'B001HBHNHE'")
item_1 = transformed.filter("item = 'B001T7QJ9O'")

item_0_a = item_0.alias("item_0_a")
item_1_a = item_1.alias("item_1_a")

item_0_a.join(item_1_a, col('item_0_a.reviewerID') == col('item_0_a.reviewerID')).show()

+----------+------+--------------+----------+----------------+----------+------+--------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index| item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+--------------+----------+----------------+----------+------+--------------+----------+----------------+
B001HBHNHE| 5.0|A1EXGL6L0QQ0M5| 0.0| 262.0|B001T7QJ9O| 5.0|A1EXGL6L0QQ0M5| 1.0| 262.0|
B001HBHNHE| 5.0| AV6A5Y2VRPETD| 0.0| 31021.0|B001T7QJ9O| 5.0| AV6A5Y2VRPETD| 1.0| 31021.0|
B001HBHNHE| 5.0|A3S5E00AAJC446| 0.0| 1560.0|B001T7QJ9O| 5.0|A3S5E00AAJC446| 1.0| 1560.0|
B001HBHNHE| 5.0|A3EWNT4ZIRF9UT| 0.0| 3549.0|B001T7QJ9O| 5.0|A3EWNT4ZIRF9UT| 1.0| 3549.0|
+----------+------+--------------+----------+----------------+----------+------+--------------+----------+----------------+

In [0]:
transformed.filter("reviewerID_index = '262'").show()
transformed.filter("reviewerID_index = '31021'").show()
transformed.filter("reviewerID_index = '1560'").show()
transformed.filter("reviewerID_index = '3549'").show()

+----------+------+--------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+--------------+----------+----------------+
B000BTHXUK| 4.0|A1EXGL6L0QQ0M5| 16270.0| 262.0|
B001CPFPJ8| 5.0|A1EXGL6L0QQ0M5| 6055.0| 262.0|
B001CZJOSQ| 5.0|A1EXGL6L0QQ0M5| 18018.0| 262.0|
B0028ZNX68| 4.0|A1EXGL6L0QQ0M5| 1786.0| 262.0|
B003O6JKLC| 5.0|A1EXGL6L0QQ0M5| 171.0| 262.0|
B00D96BO7K| 4.0|A1EXGL6L0QQ0M5| 22707.0| 262.0|
B000Q04NKW| 4.0|A1EXGL6L0QQ0M5| 320.0| 262.0|
B001HBHNHE| 5.0|A1EXGL6L0QQ0M5| 0.0| 262.0|
B001QKXJJC| 5.0|A1EXGL6L0QQ0M5| 6145.0| 262.0|
B001T7QJ9O| 5.0|A1EXGL6L0QQ0M5| 1.0| 262.0|
B003ISKM9K| 4.0|A1EXGL6L0QQ0M5| 331.0| 262.0|
B003K7X5AC| 5.0|A1EXGL6L0QQ0M5| 12825.0| 262.0|
B0047T4J20| 4.0|A1EXGL6L0QQ0M5| 278.0| 262.0|
+----------+------+--------------+----------+----------------+

+----------+------+-------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+-------------+----------+----------------+
B001HBHNHE| 5.0|AV6A5Y2VRPETD| 0.0| 31021.0|
B001T7QJ9O| 5.0|AV6A5Y2VRPETD| 1.0| 31021.0|
+----------+------+-------------+----------+----------------+

+----------+------+--------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+--------------+----------+----------------+
B000MIWE68| 5.0|A3S5E00AAJC446| 25601.0| 1560.0|
B001HBHNHE| 5.0|A3S5E00AAJC446| 0.0| 1560.0|
B001QVYT72| 5.0|A3S5E00AAJC446| 512.0| 1560.0|
B001T7QJ9O| 5.0|A3S5E00AAJC446| 1.0| 1560.0|
B004OUHKWS| 5.0|A3S5E00AAJC446| 9099.0| 1560.0|
B004QUKJQ0| 5.0|A3S5E00AAJC446| 13337.0| 1560.0|
B007DN3AVI| 5.0|A3S5E00AAJC446| 21485.0| 1560.0|
+----------+------+--------------+----------+----------------+

+----------+------+--------------+----------+----------------+
 item|rating| reviewerID|item_index|reviewerID_index|
+----------+------+--------------+----------+----------------+
B000G3C5J0| 5.0|A3EWNT4ZIRF9UT| 2880.0| 3549.0|
B001HBHNHE| 5.0|A3EWNT4ZIRF9UT| 0.0| 3549.0|
B001T7QJ9O| 5.0|A3EWNT4ZIRF9UT| 1.0| 3549.0|
B003841ZBS| 4.0|A3EWNT4ZIRF9UT| 480.0| 3549.0|
B004SKCUME| 5.0|A3EWNT4ZIRF9UT| 369.0| 3549.0|
+----------+------+--------------+----------+----------------+

In [0]:
#Recommendation list for user 262
recommend = model.recommendForAllUsers(10)
recommend_262 = recommend.filter(recommend.reviewerID_index == "262")
recommend_262 = recommend_262.toPandas()
pd.set_option('display.max_colwidth', -1)
recommend_262

,reviewerID_index,recommendations
0,262,"[(18696, 9.23350715637207), (19871, 9.138558387756348), (18559, 8.91126537322998), (3924, 8.356781959533691), (2868, 8.314875602722168), (5648, 8.288540840148926), (10424, 8.111278533935547), (29006, 8.094313621520996), (20584, 8.070418357849121), (1649, 8.037528038024902)]"


In [0]:
#Recommendation list for user 31021
recommend = model.recommendForAllUsers(10)
recommend_31021 = recommend.filter(recommend.reviewerID_index == "31021")
recommend_31021 = recommend_31021.toPandas()
pd.set_option('display.max_colwidth', -1)
recommend_31021

,reviewerID_index,recommendations
0,31021,"[(18696, 9.333854675292969), (18559, 9.202361106872559), (2868, 9.099369049072266), (9678, 9.02236557006836), (3924, 9.015478134155273), (13454, 9.012986183166504), (5606, 8.955412864685059), (10819, 8.93883991241455), (19846, 8.938278198242188), (22966, 8.913759231567383)]"


In [0]:
#Recommendation list for user 1560
recommend = model.recommendForAllUsers(10)
recommend_1560 = recommend.filter(recommend.reviewerID_index == "1560")
recommend_1560 = recommend_1560.toPandas()
pd.set_option('display.max_colwidth', -1)
recommend_1560

,reviewerID_index,recommendations
0,1560,"[(5606, 9.2345609664917), (10819, 9.110645294189453), (5648, 9.037565231323242), (19871, 8.993192672729492), (18696, 8.937543869018555), (22966, 8.925411224365234), (2868, 8.906685829162598), (16623, 8.88484001159668), (13454, 8.852377891540527), (3924, 8.85135269165039)]"


In [0]:
#Recommendation list for user 3549
recommend = model.recommendForAllUsers(10)
recommend_3549 = recommend.filter(recommend.reviewerID_index == "3549")
recommend_3549 = recommend_3549.toPandas()
pd.set_option('display.max_colwidth', -1)
recommend_3549

,reviewerID_index,recommendations
0,3549,"[(18696, 9.078761100769043), (19871, 9.066671371459961), (20584, 8.975422859191895), (18559, 8.911794662475586), (4687, 8.714488983154297), (3924, 8.555971145629883), (7041, 8.475933074951172), (19846, 8.474696159362793), (9526, 8.466764450073242), (2868, 8.464468955993652)]"


In [0]:
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

Popularity Recommender model

In [0]:
#import the reqired libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/databricks/python/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
 warnings.warn(msg, category=FutureWarning)

In [0]:
# drop not needed columns
df_new = df_output.drop("_corrupt_record", "cleaned_words", "label", "features", "indexedLabel", "probability", "rawFeatures", "rawPrediction", "words", 'reviewText','reviewTime','reviewerName','unixReviewTime','summary')
df_new_output = df_new.selectExpr("asin as item", "prediction as comment", "pre_label as rating", "reviewerID")
df_new_output.count()

Out[19]: 125210

In [0]:
df_new_output.describe().show()

+-------+--------------------+-------------------+-----------------+--------------------+
summary| item| comment| rating| reviewerID|
+-------+--------------------+-------------------+-----------------+--------------------+
 count| 125210| 125210| 125210| 125210|
 mean| 2.490072014274827E9| 0.9967255011580545|4.423312834438144| null|
 stddev|2.5459666225318317E9|0.05712969950829615|1.041619473207905| null|
 min| 0439893577| 0.0| 1.0|A00046902LP5YSDV0...|
 max| B00LFPS0CY| 1.0| 5.0| AZZYW4YOE1B6E|
+-------+--------------------+-------------------+-----------------+--------------------+

In [0]:
#For popularity recommender
#Without filtering 
df_pd = df_new_output.toPandas()
df_pd

,item,comment,rating,reviewerID
0,B00002R28C,1.0,4.0,A103B6MQ5IF2BK
1,B00004SVRR,1.0,4.0,A103QIFFX532JU
2,B00004SVXA,1.0,5.0,A105GCSUC8LQ1Z
3,B00004TJCL,1.0,1.0,A105GWGM7PDAI2
4,B00004YRQA,1.0,5.0,A105GWGM7PDAI2
...,...,...,...,...
125205,B00FHB2XB6,1.0,4.0,AYOMAHLWRQHUG
125206,B00F5Y4C7I,1.0,5.0,AYV43L6DGMH5C
125207,B00IVFCGKS,1.0,4.0,AZMISC9FAUBIR
125208,B00FOEHUTQ,1.0,5.0,AZOCGY2PBYHCI


In [0]:
df_negative = df_new_output.filter(df_new_output.comment==0)
df_pd_negative = df_negative.toPandas()
df_pd_negative

,item,comment,rating,reviewerID
0,B00004SVMX,0.0,1.0,A1COW0UIB5HA9J
1,B000034DBS,0.0,2.0,A1IQV6AI7L9NOK
2,B00000IODY,0.0,2.0,A1LBAC84TLIGAX
3,B00000JHPT,0.0,1.0,A1LBAC84TLIGAX
4,B000035XPH,0.0,1.0,A1LBAC84TLIGAX
...,...,...,...,...
405,B004LOKRDG,0.0,1.0,ADWXLAKKZP2JC
406,B0074H49NG,0.0,2.0,A25IRWH03ZHRRE
407,B009F7OUPK,0.0,1.0,A2YV6F6DLENNTM
408,B00C6Q9Y2G,0.0,1.0,AX4L6DVN3E0XX


Without filtering

In [0]:
#Count of user_id for each unique product as recommendation score 
train_data_grouped = df_pd.groupby('item').agg({'reviewerID': 'count'}).reset_index()
train_data_grouped.rename(columns = {'reviewerID': 'score'},inplace=True)
train_data_grouped

,item,score
0,0439893577,1
1,048645195X,2
2,0545496470,3
3,0615444172,3
4,0670010936,1
...,...,...
33857,B00L1N7FDK,5
33858,B00L4ZCPRG,1
33859,B00LA12PNI,1
33860,B00LBI9BKA,6


In [0]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'item'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations

,item,score,rank
14126,B001HBHNHE,211,1.0
15248,B001T7QJ9O,156,2.0
31430,B00BGA9WK2,146,3.0
9693,B000S5ODN2,132,4.0
11643,B0015AARJI,119,5.0


In [0]:
# Use popularity based recommender model to make predictions
def recommend(reviewerID):     
    user_recommendations = popularity_recommendations 
          
    #Add reviewerID column for which the recommendations are being generated 
    user_recommendations['reviewerID'] = reviewerID
      
    #Bring reviewerID column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations

In [0]:
find_recom = [10,50,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n")

The list of recommendations for the userId: 10

 reviewerID item score rank
14126 10 B001HBHNHE 211 1.0
15248 10 B001T7QJ9O 156 2.0
31430 10 B00BGA9WK2 146 3.0
9693 10 B000S5ODN2 132 4.0
11643 10 B0015AARJI 119 5.0


The list of recommendations for the userId: 50

 reviewerID item score rank
14126 50 B001HBHNHE 211 1.0
15248 50 B001T7QJ9O 156 2.0
31430 50 B00BGA9WK2 146 3.0
9693 50 B000S5ODN2 132 4.0
11643 50 B0015AARJI 119 5.0


The list of recommendations for the userId: 100

 reviewerID item score rank
14126 100 B001HBHNHE 211 1.0
15248 100 B001T7QJ9O 156 2.0
31430 100 B00BGA9WK2 146 3.0
9693 100 B000S5ODN2 132 4.0
11643 100 B0015AARJI 119 5.0


The list of recommendations for the userId: 150

 reviewerID item score rank
14126 150 B001HBHNHE 211 1.0
15248 150 B001T7QJ9O 156 2.0
31430 150 B00BGA9WK2 146 3.0
9693 150 B000S5ODN2 132 4.0
11643 150 B0015AARJI 119 5.0

In [0]:
#Count first recommendation with negative comment
data = df_pd_negative.groupby('item').agg({'reviewerID': 'count'}).reset_index()
data.rename(columns = {'reviewerID': 'score'},inplace=True)
data.sort_values(by='score')


,item,score
0,1589944968,1
255,B003XJ35GG,1
254,B003VWGBCK,1
253,B003VUO6H4,1
252,B003VJP92Q,1
...,...,...
90,B000G0HJ3K,3
330,B007XVTR3K,3
3,B0000296O5,3
22,B00005TNI6,3


With filtering

In [0]:
#With filtering
df_pd_new = df_new_output.filter(df_new_output.comment == 1)
df_pd_new = df_pd_new.toPandas()
df_pd_new

,item,comment,rating,reviewerID
0,B00002R28C,1.0,4.0,A103B6MQ5IF2BK
1,B00004SVRR,1.0,4.0,A103QIFFX532JU
2,B00004SVXA,1.0,5.0,A105GCSUC8LQ1Z
3,B00004TJCL,1.0,1.0,A105GWGM7PDAI2
4,B00004YRQA,1.0,5.0,A105GWGM7PDAI2
...,...,...,...,...
124795,B00FHB2XB6,1.0,4.0,AYOMAHLWRQHUG
124796,B00F5Y4C7I,1.0,5.0,AYV43L6DGMH5C
124797,B00IVFCGKS,1.0,4.0,AZMISC9FAUBIR
124798,B00FOEHUTQ,1.0,5.0,AZOCGY2PBYHCI


In [0]:
#Split the data randomnly into train and test datasets into 70:30 ratio
#train_data, test_data = train_test_split(
    #df_pd_new, test_size = 0.3, random_state=0)
#train_data.head()

In [0]:
#Count of user_id for each unique product as recommendation score 
train_data_grouped = df_pd_new.groupby('item').agg({'reviewerID': 'count'}).reset_index()
train_data_grouped.rename(columns = {'reviewerID': 'score'},inplace=True)
train_data_grouped

,item,score
0,0439893577,1
1,048645195X,2
2,0545496470,3
3,0615444172,3
4,0670010936,1
...,...,...
33824,B00L1N7FDK,5
33825,B00L4ZCPRG,1
33826,B00LA12PNI,1
33827,B00LBI9BKA,6


In [0]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'item'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations

,item,score,rank
14110,B001HBHNHE,211,1.0
15232,B001T7QJ9O,156,2.0
31399,B00BGA9WK2,146,3.0
9681,B000S5ODN2,132,4.0
11630,B0015AARJI,119,5.0


In [0]:
find_recom = [10,50,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n")

The list of recommendations for the userId: 10

 reviewerID item score rank
14110 10 B001HBHNHE 211 1.0
15232 10 B001T7QJ9O 156 2.0
31399 10 B00BGA9WK2 146 3.0
9681 10 B000S5ODN2 132 4.0
11630 10 B0015AARJI 119 5.0


The list of recommendations for the userId: 50

 reviewerID item score rank
14110 50 B001HBHNHE 211 1.0
15232 50 B001T7QJ9O 156 2.0
31399 50 B00BGA9WK2 146 3.0
9681 50 B000S5ODN2 132 4.0
11630 50 B0015AARJI 119 5.0


The list of recommendations for the userId: 100

 reviewerID item score rank
14110 100 B001HBHNHE 211 1.0
15232 100 B001T7QJ9O 156 2.0
31399 100 B00BGA9WK2 146 3.0
9681 100 B000S5ODN2 132 4.0
11630 100 B0015AARJI 119 5.0


The list of recommendations for the userId: 150

 reviewerID item score rank
14110 150 B001HBHNHE 211 1.0
15232 150 B001T7QJ9O 156 2.0
31399 150 B00BGA9WK2 146 3.0
9681 150 B000S5ODN2 132 4.0
11630 150 B0015AARJI 119 5.0